In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
data = pd.read_csv('iris.csv', index_col=None)

In [32]:
data

,5.1,3.5,1.4,0.2,1
0,4.9,3.0,1.4,0.2,1
1,4.7,3.2,1.3,0.2,1
2,4.6,3.1,1.5,0.2,1
3,5.0,3.6,1.4,0.2,1
4,5.4,3.9,1.7,0.4,1
...,...,...,...,...,...
8994,4.8,3.1,1.6,0.2,1
8995,5.4,3.4,1.5,0.4,1
8996,5.2,4.1,1.5,0.1,1
8997,5.5,4.2,1.4,0.2,1


In [42]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion='gini')
x = data.iloc[: , :4]
y =pd.Series(map(str, data.iloc[: , 4]))
y

0       1
1       1
2       1
3       1
4       1
       ..
8994    1
8995    1
8996    1
8997    1
8998    1
Length: 8999, dtype: object

In [45]:
800/20

40.0

In [44]:
%%timeit -r 4 -n 10
clf.fit(x, y)

17.9 ms ± 1.48 ms per loop (mean ± std. dev. of 4 runs, 10 loops each)


In [7]:
1500*2.5e-6

0.0037500000000000003

In [ ]:
def transform_milisec(sec):
    try:
        time = float(sec[:-2])
        unit = sec[-2:]
        if unit == 'ms':
            return time
        elif unit == 'ns':
            return time/1000000
        elif unit == 'µs':
            return time/1000
        else:
            return time
    except:
        print(sec[:-1])
        time = float(sec[:-1])
        
        return time*1000

In [ ]:
total_time = data.iloc[-1]['function']
data = data.iloc[:-1]
total_time

In [ ]:
data

In [ ]:
data['time'] = data['time'].apply(transform_milisec)

In [ ]:
data

In [ ]:
data['time'].sum()/1000

In [ ]:
import numpy as np



In [ ]:
arr = pd.read_csv('iris.csv')

In [ ]:
y = arr['1']
y = [1] + y.to_list()


In [63]:
y = [0]*90000

In [66]:
%%timeit -r 10 -n 10
li = []
for i in y:
    if i not in li:
        li.append(i)


2.83 ms ± 380 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [67]:
import numpy as np
y_ = np.array(y)

In [68]:
%%timeit -r 10 -n 100
np.unique(y_)

1.09 ms ± 124 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
